In [ ]:
from pynq.overlays.base import BaseOverlay
from pynq.lib.video import *
import cv2
import numpy as np
import time
 
base = BaseOverlay("base.bit")
 
# monitor configuration: 640*480 @ 60Hz,在此我们设置HDMI输出为640*480
Mode = VideoMode(640,480,24)
hdmi_out = base.video.hdmi_out
hdmi_out.configure(Mode,PIXEL_BGR)
hdmi_out.start()
 
# camera (input) configuration
frame_in_w = 640
frame_in_h = 480

 
# num_frames = 1000
readError = 0
 
globalimage=np.zeros((640,480,3), np.uint8) 
videoIn = cv2.VideoCapture(0)
videoIn.set(cv2.CAP_PROP_FRAME_WIDTH, frame_in_w);
videoIn.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_in_h);
print("capture device is open: " + str(videoIn.isOpened()))

face_cascade = cv2.CascadeClassifier(
    '/home/xilinx/jupyter_notebooks/base/video/data/'
    'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(
    '/home/xilinx/jupyter_notebooks/base/video/data/'
    'haarcascade_eye.xml')
 
while True:   
    # read next image
    start = time.time()
    ret, frame_vga = videoIn.read()
    globalimage=frame_vga
 
    if (ret):        
        np_frame = frame_vga
        gray = cv2.cvtColor(np_frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)
        for (x,y,w,h) in faces:
            cv2.rectangle(np_frame,(x,y),(x+w,y+h),(255,0,0),2)
            roi_gray = gray[y:y+h, x:x+w]
            roi_color = np_frame[y:y+h, x:x+w]
        
        #eyes = eye_cascade.detectMultiScale(roi_gray)
        #for (ex,ey,ew,eh) in eyes:
        #    cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)
        
        outframe = hdmi_out.newframe()
        outframe[:] = np_frame
        hdmi_out.writeframe(outframe)
        #print("test ok")
    else:
        readError += 1
    end = time.time()
    seconds=end-start
    #num_frames -=1
    #if num_frames<10:
    #    break

Frequency: 75600000
capture device is open: True
